# pyheatmy Demo Notebook

This notebook aims to present the various features of `pyheatmy`. It explains how to:
- create a `Column` object from an easy reading of dates
- execute the direct model, for homogeneous (section 2)  and heterogeneous (section 3) riverbeds
- execute the MCMC
- retrieve and display the various results produced during the executions of the direct model or the MCMC

This notebook doesn't provide yet information on the DREAM method implemented in 2023. For that purpose please refer to the `DREAM_VX.ipynb` notebooks

`pyheatmy` is built around the monolithic `Column` class in `core.py`. It can be executed from this class. Calculation, data retrieval, and plotting are methods provided by the `Column` class.

It is based on real data, which can be found in the `data` folder.

We recommend reading the API for more details. 

In [ ]:
from pyheatmy import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import namedtuple
NBCHAINS = 10
DELTA = 4
NCR = 3
NBITERMCMC = 35
NBBURNING = 20
DEBUGRUB = 2. # Gelman Rubin criteria
NBCELLS = 40

## 1. Chargement des données et définition d'un objet ``Column``

On doit d'abord récupérer les données issues des capteurs, qui se trouvent dans le dossier ``data``.

In [ ]:
capteur_riviere = pd.read_csv("./data/Point034_processed/processed_pressures.csv", sep = ',', names = ['dates', 'tension', 'temperature_riviere'], skiprows=1)
capteur_ZH = pd.read_csv("./data/Point034_processed/processed_temperatures.csv", sep = ',', names = ['dates', 'temperature_10', 'temperature_20', 'temperature_30', 'temperature_40'], skiprows=1)
etalonage_capteur_riv = pd.read_csv("./configuration/pressure_sensors/P508.csv")

On définit une fonction pour convertir les dates des dataframe, cela peut éviter certains problèmes.

In [ ]:
def convertDates(df: pd.DataFrame):
    """
    Convert dates from a list of strings by testing several different input formats
    Try all date formats already encountered in data points
    If none of them is OK, try the generic way (None)
    If the generic way doesn't work, this method fails
    (in that case, you should add the new format to the list)
    
    This function works directly on the giving Pandas dataframe (in place)
    This function assumes that the first column of the given Pandas dataframe
    contains the dates as characters string type
    
    For datetime conversion performance, see:
    See https://stackoverflow.com/questions/40881876/python-pandas-convert-datetime-to-timestamp-effectively-through-dt-accessor
    """
    formats = ("%m/%d/%y %H:%M:%S", "%m/%d/%y %I:%M:%S %p",
               "%d/%m/%y %H:%M",    "%d/%m/%y %I:%M %p",
               "%m/%d/%Y %H:%M:%S", "%m/%d/%Y %I:%M:%S %p", 
               "%d/%m/%Y %H:%M",    "%d/%m/%Y %I:%M %p",
               "%y/%m/%d %H:%M:%S", "%y/%m/%d %I:%M:%S %p", 
               "%y/%m/%d %H:%M",    "%y/%m/%d %I:%M %p",
               "%Y/%m/%d %H:%M:%S", "%Y/%m/%d %I:%M:%S %p", 
               "%Y/%m/%d %H:%M",    "%Y/%m/%d %I:%M %p",
               None)
    times = df[df.columns[0]]
    for f in formats:
        try:
            # Convert strings to datetime objects
            new_times = pd.to_datetime(times, format=f)
            # Convert datetime series to numpy array of integers (timestamps)
            new_ts = new_times.values.astype(np.int64)
            # If times are not ordered, this is not the appropriate format
            test = np.sort(new_ts) - new_ts
            if np.sum(abs(test)) != 0 :
                #print("Order is not the same")
                raise ValueError()
            # Else, the conversion is a success
            #print("Found format ", f)
            df[df.columns[0]] = new_times
            return
        
        except ValueError:
            #print("Format ", f, " not valid")
            continue
    
    # None of the known format are valid
    raise ValueError("Cannot convert dates: No known formats match your data!")

convertDates(capteur_riviere)
convertDates(capteur_ZH)

On définit les objets ``dH_measures`` et ``T_measures``, nécessaires à la création d'un objet ``Column``, qui réalisera les calculs :
- ``dH_measures`` contient les dates des mesures, les mesures de différence de charge, et les températures de la rivière.
- ``T_measures`` contient les dates des mesures, et les températures mesurées par les 4 capteurs de la tige.

In [ ]:
# set seed for reproducibility
np.random.seed(0)

# conversion des mesures de pression
intercept = float(etalonage_capteur_riv['P508'][2])
a = float(etalonage_capteur_riv['P508'][3])
b = float(etalonage_capteur_riv['P508'][4])
capteur_riviere['dH'] = (capteur_riviere['tension'].astype(float)-intercept-capteur_riviere['temperature_riviere'].astype(float)*b)/a

# conversion mesures de tempétratures
capteur_riviere['temperature_riviere'] = capteur_riviere['temperature_riviere'] + ZERO_CELSIUS
capteur_ZH['temperature_10'] = capteur_ZH['temperature_10'] + ZERO_CELSIUS
capteur_ZH['temperature_20'] = capteur_ZH['temperature_20'] + ZERO_CELSIUS
capteur_ZH['temperature_30'] = capteur_ZH['temperature_30'] + ZERO_CELSIUS
capteur_ZH['temperature_40'] = capteur_ZH['temperature_40'] + ZERO_CELSIUS

# définition des attributs de colonnes
dH_measures = list(zip(capteur_riviere['dates'],list(zip(capteur_riviere['dH'], capteur_riviere['temperature_riviere']))))
T_measures = list(zip(capteur_ZH['dates'], capteur_ZH[['temperature_10', 'temperature_20', 'temperature_30', 'temperature_40']].to_numpy()))

print(f"dH : {dH_measures}")
print(f"Tshaft : {T_measures}")


On définit maintenant l'objet ``Column`` à partir d'un dictionnaire.

In [ ]:
Zbottom = 0.4

col_dict = {
	"river_bed": 1., 
    "depth_sensors": [.1, .2, .3, Zbottom],
	"offset": .0,
    "dH_measures": dH_measures,
	"T_measures": T_measures,
    "sigma_meas_P": None,
    "sigma_meas_T": None,
    "inter_mode": 'lagrange',
    "nb_cells": NBCELLS,
}

col = Column.from_dict(col_dict)

L'objet créé a déjà un certain nombre d'attributs, certains étant initialisés. Se reporter au code ``core.py``.

## 2. Colonne homogène

Le modèle direct calcule les températures au cours du temps dans la colonne. Ensuite, on a accès aux flux, à la RMSE...

### 2.1. Modèle direct

Pour une colonne homogène, on définit un seul jeu de paramètres, par un tuple ou via la classe ``Param`` :

In [ ]:
Layer_homogenous = {
    "name": "Layer_homogenous",
    "zLow": Zbottom,
    "moinslog10IntrinK": 12,
    "n": .1,
    "lambda_s": 2,
    "rhos_cs": 4e6,
    "q": 1e-5
}

Layer_homogenous = Layer.from_dict(Layer_homogenous)

col.set_layers(Layer_homogenous)


Puis on appelle le modèle direct :

In [ ]:
col.compute_solve_transi(verbose=True)

Et on utilise les méthodes de récupération des résultats et de tracer

In [ ]:
col.plot_temperature_at_sensors()

nt = len(col._temperatures[0,:])
dplot=15
col.plot_temperatures_umbrella(round(nt/dplot))

flows = col.get_flows_solve()
unitLeg="m/s"
title="Débits"
col.plot_it_Zt(flows,title,unitLeg,1.04,2)

Calcul des écarts de température et comparaison des températures simulées et observées

In [ ]:
col.print_RMSE_at_sensor()
col.plot_compare_temperatures_sensors()

The simulations are of very pour quality. Let's inverse the data with mcmc

### 2.2 Bayesian inversion

In [ ]:
priors = {
    "Prior_moinslog10IntrinK": ((4+7, 9+7), .01), # (intervalle, sigma)
    "Prior_n": ((.01, .25), .01),
    "Prior_lambda_s": ((1, 10), .1),
    "Prior_rhos_cs": ((1e6,1e7), 1e5),
    "Prior_q": ((1e-12, 1e-4), 1e-10),
}

Layer_homogenous.set_priors_from_dict(priors)
print(Layer_homogenous.Prior_list)

In [ ]:
nb_chains=NBCHAINS

col.compute_mcmc(
    nb_iter = NBITERMCMC,
    verbose=True,
    nb_chain = nb_chains,
    nitmaxburning=NBBURNING,
)

In [ ]:
# On met à jour les paramètres de la colonne avec les paramètres qui minimisent l'erreur après la MCMC
col.get_best_layers()

In [ ]:
col.compute_solve_transi(verbose=False)
col.print_RMSE_at_sensor()
col.plot_compare_temperatures_sensors()

We see that the simulation is way closer to the measurements

In [ ]:
col.plot_all_param_pdf()
print(f"acceptance : {col._acceptance[-1]}")
col.plot_all_results()


### 2.3 Quantiles

Une MCMC calcule aussi des quantiles de températures et de débits d'eau. On peut les récupérer de la façon suivante :

In [ ]:
col.plot_darcy_flow_quantile()
col.plot_quantile_temperatures_sensors()

On peut aussi regarder les quantiles au niveau des capteurs, et comparer avec les mesures :

## 3. Multilayer Column

### 3.1. Direct Model

Pour une colonne stratifiée, on doit d'abord définir une liste d'objets ``Layer`` :

In [ ]:
Zinterface = 0.2
EPS = 1e-15


Couche_1 = {
    "name": "Couche 1",
    "zLow": Zinterface,
    "moinslog10IntrinK": 11,
    "n": .1,
    "lambda_s": 2,
    "rhos_cs": 4e6,
    "q": EPS
}

Couche_2 = {
    "name": "Couche 2",
    "zLow": Zbottom,
    "moinslog10IntrinK": 11,
    "n": .1,
    "lambda_s": 2,
    "rhos_cs": 4e6,
    "q": EPS
}

Layer1 = Layer.from_dict(Couche_1)
Layer2 = Layer.from_dict(Couche_2)

col.set_layers([Layer1, Layer2])

Puis on appelle le modèle direct :

In [ ]:
col.compute_solve_transi(verbose=False)

col.print_RMSE_at_sensor()
col.plot_compare_temperatures_sensors()
col.plot_temperature_at_sensors()
nt = len(col._temperatures[0,:])
dplot=15
col.plot_temperatures_umbrella(round(nt/dplot))
flows = col.get_flows_solve()
unitLeg="m/s"
title="Débits"
col.plot_it_Zt(flows,title,unitLeg,1.04,2)

## 3.2 Inférence bayésienne

L'inférence bayésienne va nous permettre d'estimer une distribution a posteriori pour chaque paramètre.

### 3.2.1. MCMC sans estimation de l'erreur

On peut lancer une MCMC en gardant $\sigma^2$ constant. On définit des distributions a priori pour chaque couche :

In [ ]:
dilat = 10
step = dilat*10


priors_couche_1 = {
    "Prior_moinslog10IntrinK": ((4+7, 9+7), .01), # (intervalle, sigma)
    "Prior_n": ((.001, .25), .005),
    "Prior_lambda_s": ((1, 10), .1),
    "Prior_rhos_cs": ((1e6,1e7), 1e5),
   # "Prior_q": ((1e-12,1e-4), 1e-10)
    "Prior_q": ((EPS/dilat,dilat*EPS), EPS/step)
}

priors_couche_2 = {
    "Prior_moinslog10IntrinK": ((4+7, 9+7), .01), # (intervalle, sigma)
    "Prior_n": ((.001, .25), .005),
    "Prior_lambda_s": ((1, 10), .1),
    "Prior_rhos_cs": ((1e6,1e7), 1e5),
   # "Prior_q": ((1e-12,1e-4), 1e-10)
    "Prior_q": ((EPS/dilat,dilat*EPS), EPS/step)
}

Layer1.set_priors_from_dict(priors_couche_1)
Layer2.set_priors_from_dict(priors_couche_2)


In [ ]:
col.compute_mcmc(verbose=True, nb_chain = nb_chains, nitmaxburning=NBBURNING, nb_iter = NBITERMCMC)

#### Recupération et affichage des distributions

In [ ]:
col.plot_all_param_pdf()

In [ ]:
col.get_best_layers()
col.compute_solve_transi(verbose=False)
col.plot_all_results()
col.plot_darcy_flow_quantile()
col.plot_quantile_temperatures_sensors()

#### 3.2.3 MCMC avec estimation de l'erreur

Pour lancer une MCMC avec estimation de la distribution de $\sigma^2$, il suffit d'ajouter une distribution a priori sur ce paramètre. On utilise pour cela la classe ``Prior``, qui est initialisée avec :
- un tuple pour l'intervalle dans lequel le paramètre varie
- un écart type pour la marche aléatoire
- une densité

In [ ]:
nb_chains=1

col.compute_mcmc(verbose=True, nb_chain = nb_chains, typealgo="estimesigma")

col.get_best_layers()
col.compute_solve_transi(verbose=False)
col.plot_all_param_pdf()
col.plot_all_results()
col.plot_darcy_flow_quantile()
col.plot_quantile_temperatures_sensors()

In [ ]:
if nb_chains>1:
    print(f"acceptance : {col._acceptance[-1]}")
else:
    print(f"acceptance : {col._acceptance}")

In [ ]:
len(col._quantiles_flows)

recupérer la valeur finale de sigma2 sur les obs et sa distrib


In [ ]:
sigma2=col.get_best_sigma2()
print(f"Variance of the temperature measurment error : {sigma2}°C")